# Create a Jupyter Widget with anywidget

*This notebook is heavily inspired by the "Getting Started" guide on the anywidget documentation: https://anywidget.dev/en/getting-started/*

## What is anywidget?

**anywidget** is a Python library that simplifies creating and publishing
custom [Jupyter Widgets](https://ipywidgets.readthedocs.io/en/latest/).
No messy build configuration or complicated cookiecutter templates.

It is <u>**not**</u> a new interactive widgets framework, but rather
an abstraction for creating custom Jupyter Widgets using modern web standards.

## Key features

- 🛠️ Create widgets **without complicated cookiecutter templates**
- 📚 **Publish to PyPI** like any other Python package
- 🤖 Prototype **within** `.ipynb` or `.py` files
- 🚀 Run in **Jupyter**, **JupyterLab**, **Google Colab**, **VSCode**, and more
- ⚡ Develop with **instant HMR**, like modern web frameworks


First you need to install `anywidget`:

In [ ]:
%pip install anywidget ipywidgets==8.1.2

Then you can define a new widget and provide the `count`, `_esm` and `_css` attributes.

In [ ]:
import anywidget
import traitlets

class CounterWidget(anywidget.AnyWidget):
    _esm = """
    function render({ model, el }) {
      let getCount = () => model.get("count");
      let button = document.createElement("button");
      button.classList.add("counter-button");
      button.innerHTML = `count is ${getCount()}`;
      button.addEventListener("click", () => {
        model.set("count", getCount() + 1);
        model.save_changes();
      });
      model.on("change:count", () => {
        button.innerHTML = `count is ${getCount()}`;
      });
      el.appendChild(button);
    }
	export default { render };
    """
    _css="""
    .counter-button { background-color: #ea580c; }
    .counter-button:hover { background-color: #9a3412; }
    """
    count = traitlets.Int(0).tag(sync=True)

counter = CounterWidget()
counter.count = 42
counter

- `count` is a stateful property for that both the client JavaScript and Python have access to.
  Shared state is defined via [traitlets](https://traitlets.readthedocs.io/en/stable/) with the `sync=True`
  keyword argument.

- `_esm` specifies a <u>**required**</u> [ECMAScript module](https://nodejs.org/api/esm.html) for the widget.
  It defines and exports `render`, a function for rendering and initializes dynamic updates for the custom widget.

- `_css` specifies an <u>**optional**</u> CSS stylesheet to load for the widget. It can be a full URL or plain text. Styles are loaded
  in the global scope if using this feature, so take care to avoid naming conflicts.

  Feel free to modify some of the code above and re-execute the cells to see the changes 🪄

## Progressive Development

As your widgets grow in complexity, it is recommended to separate your
front-end code from your Python code. Just move the `_esm` and `_css`
definitions to separate files and reference them via path.

In [ ]:
from pathlib import Path

class CounterWidget(anywidget.AnyWidget):
    _esm = Path('/drive/index.js')
    _css= Path('/drive/index.css')
    count = traitlets.Int(0).tag(sync=True)

counter = CounterWidget()
counter.count = 42
counter

**Note**: since this particular notebook is meant to be used in JupyterLite, we specify `/drive` as the prefix for finding the `.js` and `.css` files. `/drive` is the location of the underlying (in-browser) file system where JupyterLite expects to find the files, so they can be displayed in the file browser.

You can now open `index.js` and `index.css` in JupyterLab and edit the files directly. After making changes, you will have to recreate the widget so they are applied.